# Amazon Bedrock AgentCore RuntimeでMCPサーバーをホスティングする

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore RuntimeでMCP(Model Context Protocol)サーバーをホスティングする方法を学びます。Amazon Bedrock AgentCore Python SDKを使用して、MCPツールをAmazon Bedrock AgentCoreと互換性のあるMCPサーバーとしてラップします。

Amazon Bedrock AgentCore Python SDKは、MCPサーバーの実装の詳細を処理するため、ツールのコア機能に集中できます。コードをAgentCoreの標準化されたMCPプロトコル契約に変換し、直接通信できるようにします。

### チュートリアルの詳細

| 情報                | 詳細                                                      |
|:--------------------|:----------------------------------------------------------|
| チュートリアルの種類 | ツールのホスティング                                      |
| ツールの種類         | MCPサーバー                                              |
| チュートリアルの構成要素 | AgentCoreランタイムでMCPサーバーをホスティング           |
| チュートリアルの分野 | 分野横断                                                 |
| 例の複雑さ           | 簡単                                                     |
| 使用するSDK          | Amazon BedrockAgentCore Python SDKとMCP                 |

### チュートリアルのアーキテクチャ

このチュートリアルでは、MCPサーバーをAgentCoreランタイムにデプロイする方法を説明します。

デモンストレーション目的で、`add_numbers`、`multiply_numbers`、`greet_user`の3つのツールを持つ簡単なMCPサーバーを使用します。

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="40%"/>
</div>

### チュートリアルの主な機能

* カスタムツールを持つMCPサーバーの作成
* MCPサーバーをローカルでテスト
* Amazon Bedrock AgentCore RuntimeでMCPサーバーをホスティング
* 認証を使ってデプロイされたMCPサーバーを呼び出す

## 前提条件

このチュートリアルを実行するには、以下が必要です:

* Python 3.10+
* 設定済みの AWS 認証情報
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) ライブラリ
* 動作中の Docker

In [ ]:
#!uv add -r requirements.txt --active

日本語訳:

このコマンドは、 requirements.txt ファイルに記載された依存関係をインストールするために使用されます。 --active オプションは、現在のアクティブな仮想環境にパッケージをインストールすることを指定しています。

## MCPの理解 (Model Context Protocol)

MCPは、AIモデルが外部データやツールに安全にアクセスできるようにするプロトコルです。主要な概念は次のとおりです。

* **Tools**: AIが呼び出して操作を実行できる関数
* **Streamable HTTP**: AgentCore Runtimeで使用される転送プロトコル
* **Session Isolation**: `Mcp-Session-Id` ヘッダーを介して、各クライアントに分離されたセッションが提供される
* **Stateless Operation**: スケーラビリティのため、サーバーはステートレス操作をサポートする必要がある

AgentCore Runtimeは、デフォルトのパスとして `0.0.0.0:8000/mcp` でMCPサーバーがホストされることを期待しています。

### プロジェクトの構造

適切な構造でプロジェクトを設定しましょう。

```
mcp_server_project/
├── mcp_server.py              # メインのMCPサーバーコード
├── my_mcp_client.py           # ローカルテスト用クライアント
├── my_mcp_client_remote.py    # リモートテスト用クライアント
├── requirements.txt           # 依存関係
└── __init__.py                # Pythonパッケージマーカー
```

## MCP サーバーの作成

さて、3 つの簡単なツールを使って MCP サーバーを作成しましょう。このサーバーは `stateless_http=True` の FastMCP を使用しており、これは AgentCore Runtime との互換性が必要とされています。

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """ 2 つの数値を足し合わせる """
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """ 2 つの数値を掛け合わせる """
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """ ユーザーに名前で挨拶する """
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

以下が日本語訳になります。

### このコードの機能

* **FastMCP**: ツールをホストできる MCP サーバーを作成します
* **@mcp.tool()**: Python 関数を MCP ツールに変換するデコレーター
* **stateless_http=True**: AgentCore Runtime との互換性に必要です
* **Tools**: 異なる種類の操作を示す 3 つの簡単なツール

## ローカルテスト用クライアントの作成

AgentCore Runtime にデプロイする前に、MCP サーバーをローカルでテストするためのクライアントを作成しましょう。

In [ ]:
%%writefile my_mcp_client.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### ローカルでのテスト

MCP サーバーをローカルでテストするには:

1. **ターミナル 1**: MCP サーバーを起動
   ```bash
   python mcp_server.py
   ```

2. **ターミナル 2**: テストクライアントを実行
   ```bash
   python my_mcp_client.py
   ```

出力に 3 つのツールが表示されるはずです。

## 認証のための Amazon Cognito の設定

AgentCore Runtime は認証が必要です。デプロイした MCP サーバーにアクセスするための JWT トークンを提供するために、Amazon Cognito を使用します。

In [ ]:
import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '.')
utils_dir = os.path.abspath(utils_dir)

sys.path.insert(0, utils_dir)

from utils import create_agentcore_role, setup_cognito_user_pool

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

## ステップ 5: IAM 実行ロールを作成する

始める前に、AgentCore Runtime 用の IAM ロールを作成しましょう。このロールは、ランタイムが動作するために必要な許可を提供します。

In [ ]:
tool_name = "mcp_server"
print(f"Creating IAM role for {tool_name}...")
agentcore_iam_role = create_agentcore_role(agent_name=tool_name)
print(f"IAM role created ✓")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

## AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、および要件ファイルで AgentCore Runtime デプロイを設定します。また、スターターキットを設定して、起動時に Amazon ECR リポジトリを自動作成するようにします。

設定ステップでは、アプリケーションコードに基づいて Dockerfile が生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

## MCP サーバーを AgentCore Runtime に起動する

ここまでで Dockerfile を作成できました。次は MCP サーバーを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## AgentCore ランタイムの状態を確認する

AgentCore ランタイムをデプロイしたので、次はそのデプロイ状況を確認し、準備が整うのを待ちましょう。

```
kubectl get pods -n agentcore
```

出力例:

```
NAME                                    READY   STATUS    RESTARTS   AGE
agentcore-controller-7d4459c6c8-8qlzm   1/1     Running   0          9m58s
agentcore-runtime-5b9f6d5d5d-6jqrv      1/1     Running   0          9m58s
agentcore-runtime-5b9f6d5d5d-jbhwn      1/1     Running   0          9m58s
agentcore-runtime-5b9f6d5d5d-pxqxj      1/1     Running   0          9m58s
```

すべての Pod が `Running` 状態になるまで待ちます。

In [ ]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## リモートアクセスの設定を保存する

デプロイした MCP サーバーを呼び出す前に、Agent ARN と Cognito の設定を AWS Systems Manager Parameter Store と AWS Secrets Manager に保存しておきましょう。これで簡単に取得できるようになります。

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## リモートテスト用クライアントの作成

次に、デプロイした MCP サーバーをテストするためのクライアントを作成しましょう。このクライアントは AWS から必要な認証情報を取得し、デプロイされたサーバーに接続します。

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

以下が日本語訳になります。

## デプロイした MCP サーバーのテスト

リモートクライアントを使って、デプロイした MCP サーバーをテストしましょう:

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

以下が日本語訳になります。

## MCP ツールをリモートで呼び出す

ここでは、ツールを一覧表示するだけでなく、MCP の完全な機能を実演するためにそれらを呼び出す拡張クライアントを作成しましょう。

```python
import os
import subprocess
import tempfile

from mcp import mcp

# MCP クライアントを作成
client = mcp.Client()

# 利用可能なツールを一覧表示
tools = client.list_tools()
print(f"Available tools: {', '.join(tools)}")

# ツールを呼び出す
tool_name = "gpt-3.5-turbo"
prompt = "Hello, how are you?"
response = client.invoke(
    tool_name=tool_name,
    prompt=prompt,
)
print(f"{tool_name} response: {response}")

# ツールを呼び出し、出力をファイルに保存
tool_name = "gpt-3.5-turbo"
prompt = "Write a short poem about nature."
with tempfile.NamedTemporaryFile(mode="w+", delete=False) as tmp:
    tmp_path = tmp.name
    response = client.invoke(
        tool_name=tool_name,
        prompt=prompt,
        output_path=tmp_path,
    )
    print(f"Output saved to: {tmp_path}")
    with open(tmp_path, "r") as f:
        print(f.read())
    os.remove(tmp_path)
```

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## テストツールの呼び出し

実際に MCP ツールを呼び出して、テストしましょう。

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## 次のステップ

AgentCore Runtime に MCP サーバーを正常にデプロイできたので、次のことができます。

1. **ツールの追加**: 追加のツールで MCP サーバーを拡張する
2. **カスタム認証**: カスタム JWT 認可者を実装する
3. **統合**: 他の AgentCore サービスと統合する

## クリーンアップ (オプション)

このチュートリアルで作成したリソースを削除したい場合は、次のセルを実行してください:

In [ ]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
iam_client = boto3.client('iam')
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    print("Deleting IAM role policies...")
    policies = iam_client.list_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
    )

    for policy_name in policies['PolicyNames']:
        iam_client.delete_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyName=policy_name
        )
    
    iam_client.delete_role(
        RoleName=agentcore_iam_role['Role']['RoleName']
    )
    print("✓ IAM role deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")

    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

# 🎉 おめでとうございます!

あなたは次の作業を無事に完了しました:

✅ **カスタムツールを使用した MCP サーバーの作成**
✅ **MCP クライアントを使用したローカルでのテスト**
✅ **Amazon Cognito を使用した認証のセットアップ**
✅ **AgentCore Runtime を使用した AWS へのデプロイ**
✅ **適切な認証を使用したリモート呼び出し**
✅ **MCP の概念とベストプラクティスの習得**

あなたの MCP サーバーは現在、Amazon Bedrock AgentCore Runtime 上で実行されており、本番環境での使用が可能です!

## 概要

このチュートリアルでは、以下の内容を学びました:
- FastMCP を使用した MCP サーバーの構築
- AgentCore 互換性のための状態レス HTTP トランスポートの設定
- Amazon Cognito を使用した JWT 認証のセットアップ
- AWS 上での MCP サーバーのデプロイと管理
- ローカルとリモートの両方でのテスト
- ツール呼び出しのための MCP クライアントの使用

デプロイされた MCP サーバーは、今後、より大規模な AI アプリケーションやワークフローに統合することができます!